<a href="https://colab.research.google.com/github/boydvcu/VCU_BNFO301_Spring2022/blob/main/Lab5_Matrix_KEY_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Creating Consensus Sequences</h1>

This program will create a consensus sequence, which represents the most frequent nucleotide at each position in an alignment of two or more sequences.  Consensus sequences can be useful for identifying and visualizing motifs in a DNA or amino acid sequence. The data is a FASTA file that contains sequences from Human, Gorilla, Dolphin, Rat and Mouse.

> Indented block




### Setup
Load the Sequence Data File.  Please run this block without changing the code.

File : https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta

In [1]:
import os.path
# Load the genbank file
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta"
DEFAULT_FILE_NAME = 'Assignment3_data.fasta'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB



--2023-12-16 18:37:42--  https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312 [text/plain]
Saving to: ‘Assignment3_data.fasta’

Assignment3_data.fa 100%[===================>]     312  --.-KB/s    in 0s      

2023-12-16 18:37:42 (22.9 MB/s) - ‘Assignment3_data.fasta’ saved [312/312]



###Step 1.  Create a definition to clean the FASTA formated file and add sequences into a list.  

We created a second list that contains the headers (with the ">" removed).  

We created function that can print the each list.  This later function will allow you to evaluate if your dictionary contains the expected information.

In [2]:
#Read in fasta and create list of sequences (sequences) and a list of sequence identifiers (seqIDs)

def ReadFile(fh):
    # Open file
    file = open(fh)

    # Initialize variables
    sequences = []  # Store sequences
    seqIds = []  # Store sequence headers

    # Loop through file
    for line in open(fh):

        # Remove all unwanted white spaces
        line = line.strip()

        # Find fasta header for first sequence
        if line.startswith(">"):
            seqIds.append(line.replace(">", ""))
        else:
            sequences.append(line)

    # Return lists
    return sequences, seqIds


# -----------------------------
# Utility function to pretty print
# ------------------------------
def printInfo(seqIds, seqs):
    for seqId, seq in zip(seqIds, seqs):
        print("{: <12} {: <20}".format(seqId, seq))


sequences, seqIds = ReadFile(fileName)
# print(sequences, seqIds)
printInfo(seqIds, sequences)





 Human       GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
 Gorilla     GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
 Dolphin     GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG
 Rat         GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC
 Mouse       GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


In [3]:

# -----------------------------
# Convert sequence to matrix for easy computation
# ------------------------------
def generateMatrix(seqs):
    # Lengths of the aligned sequences are equal
    # Get length of first sequence
    length = len(seqs[0])

    # Initialize variables
    matrix = []  # Will store all values at each position

    # Get a list of list
    # Reads fasta file as matrix
    # Example:
    #        ATGCA
    #        ATGAA
    #        TCGAT
    #             Bases at index 0   Bases at index 1 ...
    # positions = [["A", "A", "T"],  ["T", "T", "C"], ...]]

    for base in range(length):
        column = []
        for sequence in range(len(seqs)):
            column.append(seqs[sequence][base])
        matrix.append(column)

    return matrix


matrix = generateMatrix(sequences)

#printing first 3 rows to show the matrix
print(matrix[0:3])



[['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G']]


###Step-2.  Creating a function that returns a consensus sequence using the most fequent base.

1. If all bases are in agreement, return base as upper.case.  e.g. ['G', 'G', 'G', 'G', 'G'] returns G
2. If there is an equal split between bases in a column, return "n".  e.g. ['G', 'G', 'A', 'A', 'A', 'G'] returns n
3. If the the bases are not in agreement, but one base is more frequent than other, return the most frequent base in lower.case.  eg. ['G', 'G', 'G', 'G', 'A'] returns g

The output should look like this: GgaGcg?ct?GGAgCCgGacCcgGAcCcCgGCGAt?GCcGcCtGc?tCtC?


In [4]:
def getConsensus(matrix):
    # Initialize variables
    consensus = ""

    # Loop through each inner list
    for row in matrix:
        freq = {}
        for char in row:
            # Get counts for each base
            freq[char] = freq.get(char, 0) + 1

        # Get the max value
        maxFreq = max(freq, key=freq.get)

        # See if there are multiple keys with the max value
        maxKeys = [k for k, v in freq.items() if v == freq.get(maxFreq)]

        # Calculate abundance of base at position
        abundance = float(freq.get(maxFreq)) / len(row)
        if abundance == 1:
            consensus += maxKeys[0].upper()
        elif len(maxKeys) == 1 and abundance > 0.5:  # Base is common at position
            consensus += maxKeys[0].lower()
        else:
            consensus += "?"

    return consensus


consensus = getConsensus(matrix)
print("{: <25} {: <20}\n".format("CONSENSUS", consensus))


CONSENSUS                 GgaGcg?ct?GGAgCCgGacCcgGAcCcCgGCGAt?GCcGcCtGc?tCtC?



###Bonus  
One way aligned sequences are often visualized is as a sequence logo. For an additional point, go to https://weblogo.berkeley.edu/logo.cgi and upload the provided FASTA file. Save the image and upload to canvas.  You may do the bonus between class meetings on your own time.